In [ ]:
import numpy as np
import pandas as pd 

In [ ]:
from sklearn.metrics import classification_report
import autokeras as ak
import os
from tqdm import tqdm

from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from random import shuffle

In [ ]:
TRAINING_TIMES = [
    60 * 60, # 1 hour
    60 * 60 * 2 # 2 hours
]

In [ ]:
SIZE = 50

In [ ]:
PATH = '../input/cats-vs-dogs/train/'

In [ ]:
def get_label(file):
    class_label = file.split('.')[0]
    if class_label == 'dog': label_vector = 0
    elif class_label == 'cat': label_vector = 1
    return label_vector

In [ ]:
def get_data():
    data = []
    files = os.listdir(PATH)
    for image in tqdm(files):
        
        label_vector = get_label(image)
        

        img = Image.open(PATH + image).convert('L')
        img = img.resize((SIZE,SIZE))
        
        data.append([np.asarray(img),np.array(label_vector)])
        
    shuffle(data)
    return data

In [ ]:
data = get_data()

In [ ]:
train = data[:7000]
test = data[7000:]

In [ ]:
x_train = [data[0] for data in train]
y_train = [data[1] for data in train]
x_test = [data[0] for data in test]
y_test = [data[1] for data in test]

In [ ]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
x_train = np.array(x_train).reshape(-1,SIZE,SIZE,1)
x_test = np.array(x_test).reshape(-1,SIZE,SIZE,1)

In [ ]:
for seconds in TRAINING_TIMES:

    print("Training model for {} seconds max...".format(
        seconds))
    model = ak.ImageClassifier(verbose=True)
    model.fit(x_train, y_train, time_limit=seconds)
    model.final_fit(x_train, y_train, x_test, y_test, retrain=True)

    score = model.evaluate(x_test, y_test)
    predictions = model.predict(x_test)
    report = classification_report(y_test, predictions,
        target_names=['dog', 'cat'])

    f = open("{}.txt".format(seconds), "w")
    f.write(report)
    f.write("\nscore: {}".format(score))
    f.close()